---
# 1. 필요 라이브러리 설치 및 불러오기
---

In [1]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 105705, done.
remote: Counting objects: 100% (366/366), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 105705 (delta 202), reused 283 (delta 159), pack-reused 105339
Receiving objects: 100% (105705/105705), 98.15 MiB | 17.86 MiB/s, done.
Resolving deltas: 100% (78058/78058), done.


In [2]:
pip install -e


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

-e option requires 1 argument


In [3]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 28.9 MB/s 
     |████████████████████████████████| 212 kB 92.0 MB/s 
     |████████████████████████████████| 115 kB 63.5 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 141 kB 76.7 MB/s 
     |████████████████████████████████| 596 kB 64.6 MB/s 
     |████████████████████████████████| 127 kB 87.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
pip install huggingface-hub==0.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 6.2 MB/s 
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.8.1
    Uninstalling huggingface-hub-0.8.1:
      Successfully uninstalled huggingface-hub-0.8.1


In [5]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 32.5 MB/s 
     |████████████████████████████████| 6.6 MB 75.9 MB/s 


- glue/mnli 다운을 위한 업그레이드

In [6]:
pip install tensorflow-datasets -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import os
import numpy as np
from argparse import ArgumentParser
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

---
# 2. mnli 데이터셋을 분석해 보기
---

Multi-Genre Natural Language Inference Corpus는 텍스트 수반 주석이 있는 문장 쌍의 크라우드 소싱 모음입니다.  
전제(premise) 문장과 가설(hypothesis) 문장이 주어졌을 때, 과제는 전제가 가설을 수반하는지(entailment), 가설과 모순되는지(모순), 아니면 둘 다 그렇지 않은지(중립) 예측하는 것입니다.  
  
전제 문장은 전사된 연설, 소설 및 정부 보고서를 포함하여 10가지 다른 출처에서 수집됩니다.  
우리는 저자로부터 개인 레이블을 얻은 표준 테스트 세트를 사용하고 일치(도메인 내) 섹션과 일치하지 않는(교차 도메인) 섹션 모두에서 평가합니다.  

또한 SNLI 코퍼스를 보조 훈련 데이터의 550k 예시로 사용하고 권장합니다.

https://www.tensorflow.org/datasets/catalog/glue#gluemnli 

In [8]:
data, info = tfds.load('glue/mnli', with_info=True)
info.splits['train'].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/5 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/392702 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/glue/mnli/2.0.0.incomplete8MH8MK/glue-test_matched.tfrecord*...:   0%|        …

Generating test_mismatched examples...:   0%|          | 0/9847 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/glue/mnli/2.0.0.incomplete8MH8MK/glue-test_mismatched.tfrecord*...:   0%|     …

Dataset glue downloaded and prepared to ~/tensorflow_datasets/glue/mnli/2.0.0. Subsequent calls will reuse this data.


392702

In [9]:
print('총 데이터 개수:',info.splits['train'].num_examples)

총 데이터 개수: 392702


In [10]:
data['train'].take(1)

<TakeDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>

- 0 = entailment (전제가 가설을 수반)
- 1 = neutral (중립)
- 2 = contradiction (전제가 가설에 모순)

In [11]:
examples = data['train'].take(1)
for example in examples:
    premise = example['premise'].numpy()
    hypothesis = example['hypothesis'].numpy()
    label = example['label'].numpy()

    print('전제 문장: ',premise)
    print('가설 문장: ',hypothesis)    
    print('예측 값: ',label)

전제 문장:  b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'
가설 문장:  b'Meaningful partnerships with stakeholders is crucial.'
예측 값:  1


---
# 3. Processor의 활용
---

- Processor는 'Raw Dataset를 Annotated Dataset으로 변환'하는 역할

In [12]:
class DataProcessor:
    """Base class for data converters for sequence classification data sets."""

    def get_example_from_tensor_dict(self, tensor_dict):
        """
        Gets an example from a dict with tensorflow tensors.

        Args:
            tensor_dict: Keys and values should match the corresponding Glue
                tensorflow_dataset examples.
        """
        raise NotImplementedError()

    def get_train_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of :class:`InputExample` for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    def tfds_map(self, example):
        """
        Some tensorflow_datasets datasets are not formatted the same way the GLUE datasets are. This method converts
        examples to the correct format.
        """
        if len(self.get_labels()) > 1:
            example.label = self.get_labels()[int(example.label)]
        return example

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

- 원본 데이터셋을 처리하여 모델에 입력할 수 있도록 정리해 주는 클래스

In [13]:
class MnliProcessor(DataProcessor):
    """Processor for the MRPC data set (GLUE version)."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_example_from_tensor_dict(self, tensor_dict):
        """See base class."""
        return InputExample(
            tensor_dict["idx"].numpy(),
            tensor_dict["premise"].numpy().decode("utf-8"),
            tensor_dict["hypothesis"].numpy().decode("utf-8"),
            str(tensor_dict["label"].numpy()),
        )

    def get_train_examples(self, data_dir):
        """See base class."""
        print("LOOKING AT {}".format(os.path.join(data_dir, "train.tsv")))
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "dev_matched.tsv")), "dev_matched")

    def get_test_examples(self, data_dir):
        """See base class."""
        return self._create_examples(self._read_tsv(os.path.join(data_dir, "test_matched.tsv")), "test_matched")

    def get_labels(self):
        """See base class."""
        return ["contradiction", "entailment", "neutral"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            text_b = line[4]
            label = None if set_type == "test" else line[0]
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

In [14]:
processor = MnliProcessor()
examples = data['train'].take(1)

for example in examples:
    print('------원본데이터------')
    print(example)  
    example = processor.get_example_from_tensor_dict(example)
    print('------processor 가공데이터------')
    print(example)

------원본데이터------
{'hypothesis': <tf.Tensor: shape=(), dtype=string, numpy=b'Meaningful partnerships with stakeholders is crucial.'>, 'idx': <tf.Tensor: shape=(), dtype=int32, numpy=16399>, 'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'premise': <tf.Tensor: shape=(), dtype=string, numpy=b'In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.'>}
------processor 가공데이터------
InputExample(guid=16399, text_a='In recognition of these tensions, LSC has worked diligently since 1995 to convey the expectations of the State Planning Initiative and to establish meaningful partnerships with stakeholders aimed at fostering a new symbiosis between the federal provider and recipients of legal services funding.', text_b='Meaningful partnerships with stakeh

---
# 4. Tokenizer와 Model
---

In [15]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification

huggingface_tokenizer = RobertaTokenizer.from_pretrained("roberta-base", use_fast=True)
huggingface_model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# def transform(data):
#   return huggingface_tokenizer(
#       data['premise'],
#       data['hypothesis'],
#       truncation = True,
#       padding = 'max_length',
#       return_token_type_ids = False)

In [17]:
import datasets
from datasets import load_dataset

hugging_load = load_dataset('glue', 'mnli')
print(hugging_load )

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})


---
### 4-1 변환하기
---

아래 과정은 hugging_face를 통해서 데이터를 다시 로드한다면 필요 없는 과정이다.  
다만, 단순한 호기심과 오기가 생겨서 작업을 하게 되었다.

In [18]:
print(type(data)) # tensorflow_dataset load
print(type(hugging_load)) # hugging_face load

<class 'dict'>
<class 'datasets.dataset_dict.DatasetDict'>


tensorflow_dataset을 통해서 mnli를 불러온 파일과  
hugging_face를 통해서 mnli를 불러온 파일의 내용은 같으나,  
파일을 불러오는 과정에서 처리가 다르게 되어있다.  

hugging_face를 통헤서 불러오는 경우 저장한 변수명과 원하는 데이터명만 불러와주면 되게끔 처리가 잘 되어서 편하게 데이터 값이 잘 불러와 지나, 

tensorflow_dataset의 경우에는 처리가 필요하다 

    def transform(data):
      return huggingface_tokenizer(
          data['premise'],
          data['hypothesis'],
          truncation = True,
          padding = 'max_length',
          return_token_type_ids = False,
          )

위의 transform함수에 적용하기 위해서는 이전에 불러온 data라는 변수명으로는 처리가 불가능해 

굳이굳이 데이터를 처리하고 해당 함수에 넣도록 바꿔주었다.

<참고>
data 라는 변수명은 tensorflow_dataset을 통해서 불러온 값 
hugging_load라는 변수명은 hugging_face를 불러온 값


---
hugging_face dataset load

----

- data값이나 hugging_load 값의 포함된 값은 동일하나 처리되어 있는 모습이 다르다

In [19]:
data

{'test_matched': <PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 'test_mismatched': <PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 Split('train'): <PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': TensorSpec(shape=(), dtype=tf.int32, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'premise': TensorSpec(shape=(), dtype=tf.string, name=None)}>,
 'validation_matched': <PrefetchDataset element_spec={'hypothesis': TensorSpec(shape=(), dtype=tf.string, name=None), 'idx': Tenso

In [20]:
hugging_load

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

- hugging_fac을 통한 load의 경우 dict 형식으로 바로 변수명만 불러오면 사용하게 끔 잘 구성되어 있다.

In [21]:
hugging_load ['train']

Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 392702
})

- 리스트 슬라이싱 까지 가능하다

In [22]:
hugging_load ['train']['premise'][:3]

['Conceptually cream skimming has two basic dimensions - product and geography.',
 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him',
 'One of our number will carry out your instructions minutely.']

In [23]:
print(type(hugging_load ['train']['premise'][:5]))
print(type(hugging_load ['train']['premise'][1]))

<class 'list'>
<class 'str'>


---
tensorflow_dataset load

----

- tensorflow_dataset의 경우 슬라이싱이 불가능 하다

In [24]:
data['train'][:5]

TypeError: ignored

In [ ]:
examples = data['train'].take(1)
for example in examples:
  print(example)

In [ ]:
print(data)
print('='*200)
print(type(data))
print(type(data['train']))

- 결론적으로 목적은 transform 함수에 데이터셋을 넣기 위해  
2단 인덱싱이 가능하도록 딕셔너리 형태로 변환하는 것이다.

In [ ]:
convert_data = {} #  빈 딕셔너리 생성

def convert_to_list(name):
  premise = []  # 각종 딕셔너리 안에 들어갈 변수명 생성
  hypothesis = []
  idx = []
  label = []

  examples = data[name] # tensorflow_dataset을 통해 load한 데이터를 사용

  for index, example in enumerate(examples):
      premise.append((example['premise'].numpy()).decode()) # list 형식으로 저장
                      # decode() = bytes 객체를 문자열로 변경 
      hypothesis.append((example['hypothesis'].numpy()).decode())
      label.append(example['label'].numpy())
      idx.append(index)
  convert_data[name] = {'premise': premise,'hypothesis': hypothesis, 'idx':idx, 'label':label} # 딕셔너리에 추가

  return convert_data

- train은 변수명이 splits('train')으로 되어 있다

In [ ]:
name = list(data.keys())
name = ['train'] + name[1:] 
name

- 각 변수 추가

In [ ]:
for i in name:
  test1 = convert_to_list(i)

- 딕셔너리 확인

In [ ]:
convert_data['train']['premise'][:5]

In [ ]:
def transform(data):
  return huggingface_tokenizer(
      data['premise'],
      data['hypothesis'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = False,
      )
  

encoded_dataset = convert_data.map(transform, batched=True)

In [ ]:
print(type(convert_data['train'][:5]))

hugging_load의 경우 인덱스 슬라이싱이 가능하나 
convert_data는 슬라이싱이 되지 않는다. 

클래스를 통해 해당 문제를 개선해 보겠다.

In [ ]:
class convert_data_class:

  def __init__(self):
    self.convert = {}
    self.result = {}

  def make_dic(self,data, name):
    self.data = data
    self.name =  name 

    premise = []  # 각종 딕셔너리 안에 들어갈 변수명 생성
    hypothesis = []
    idx = []
    label = []

    examples = data[self.name] # tensorflow_dataset을 통해 load한 데이터를 사용

    for index, example in enumerate(examples):
        premise.append((example['premise'].numpy()).decode()) # list 형식으로 저장
                        # decode() = bytes 객체를 문자열로 변경 
        hypothesis.append((example['hypothesis'].numpy()).decode())
        label.append(example['label'].numpy())
        idx.append(index)
    self.convert[self.name] = {'premise': premise,'hypothesis': hypothesis, 'idx':idx, 'label':label} # 딕셔너리에 추가

    return self.convert
    
  def __getitem__(self, idx):

    for dic in self.convert.keys():
      self.result[dic] = self.convert[dic][idx]
    return self.result

In [ ]:
a = convert_data_class()

In [ ]:
for i in name:
  test = a.make_dic(data,i)

------

- hugging_face load한 데이터

In [ ]:
print(type(hugging_load))
print(type(hugging_load['train']))
print(type(hugging_load['train']['premise']))

- tensorflow에서 load한 데이터

In [ ]:
print(type(data))
print(type(data['train']))
print(type(data['train']['premise']))

- 변환 시도 중인 데이터

In [ ]:
print(type(test))
print(type(test['train']))
print(type(test['train']['premise']))

- dataset을 인덱싱 + 슬라이싱을 하는 과정에서 마무리를 하지 못하여 결론적으로 변환에는 실패하였다. 

- dataset을 인덱싱 + 슬라이싱을 하는 과정에서 마무리를 하지 못하여 결론적으로 변환에는 실패하였다. 

- 여러 시도를 해보면서 datasetDict로 변환하는 것은 어렵지 않음을 알 수 있었으나, 
dataset으로 변환하는 과정에서 어려움을 겪었다.

- 외부 api를 통해서 변환이 가능한 것인지, 따른 설정을 통해 변환이 가능한 것인지 확인을 하지 못하였다. 

- 다만, class를 구성하고 찾아보는 과정에서 class에 대한 이해가 한 걸음 더 늘었다고 생각한다. 

----
### 4-2 토큰화
---


In [ ]:
# encoded_dataset = hugging_load.map(transform, batched=True)

input_ids와 attention_mask가 추가로 생성이 되었다.

- 토크나이저를 이용하여 데이터를 정수화 하는 함수

In [25]:
def _glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="claasification") :
    if max_length is None :
        max_length = tokenizer.max_len
    if label_list is None:
        label_list = processor.get_labels()
        print("Using label list %s" % (label_list))

    label_map = {label: i for i, label in enumerate(label_list)}
    labels = [label_map[example.label] for example in examples]

    batch_encoding = tokenizer(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        truncation=True,
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}

        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: %s" % (example.guid))
        print("features: %s" % features[i])

    return features

- 정수화한 데이터를 Tensorflow 데이터셋으로 생성하는 함수

In [26]:
def tf_glue_convert_examples_to_features(examples, tokenizer, max_length, processor, label_list=None, output_mode="classification") :
    """
    :param examples: tf.data.Dataset
    :param tokenizer: pretrained tokenizer
    :param max_length: example의 최대 길이(기본값 : tokenizer의 max_len)
    :param task: GLUE task 이름
    :param label_list: 라벨 리스트
    :param output_mode: "regression" or "classification"

    :return: task에 맞도록 feature가 구성된 tf.data.Dataset
    """
    examples = [processor.tfds_map(processor.get_example_from_tensor_dict(example)) for example in examples]
    features = _glue_convert_examples_to_features(examples, tokenizer, max_length, processor)
    label_type = tf.int64

    def gen():
        for ex in features:
            d = {k: v for k, v in asdict(ex).items() if v is not None}
            label = d.pop("label")
            yield (d, label)

    input_names = ["input_ids"] + tokenizer.model_input_names

    return tf.data.Dataset.from_generator(
        gen,
        ({k: tf.int32 for k in input_names}, label_type),
        ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
    )

In [27]:
a = [0, 1121, 4972, 9, 209, 5734, 6, 226, 3632, 34, 1006, 28554, 187, 7969, 7, 15451, 5, 2113, 9, 5, 331, 9619, 11980, 8, 7, 5242, 6667, 8670, 19, 7193, 3448, 23, 26242, 10, 92, 39835, 38115, 227, 5, 752, 3696, 8, 11940, 9, 1030, 518, 1435, 4, 2, 2, 5096, 19266, 2650, 8670, 19, 7193, 16, 4096, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
len(a)

256

- 전체 데이터셋 구성

In [28]:
# train 데이터셋
train_dataset = tf_glue_convert_examples_to_features(data['train'], huggingface_tokenizer, max_length=256, processor=processor)
train_dataset_batch = train_dataset.shuffle(100).batch(16).repeat(2)

Using label list ['contradiction', 'entailment', 'neutral']


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

*** Example ***
guid: 16399
features: InputFeatures(input_ids=[0, 1121, 4972, 9, 209, 5734, 6, 226, 3632, 34, 1006, 28554, 187, 7969, 7, 15451, 5, 2113, 9, 5, 331, 9619, 11980, 8, 7, 5242, 6667, 8670, 19, 7193, 3448, 23, 26242, 10, 92, 39835, 38115, 227, 5, 752, 3696, 8, 11940, 9, 1030, 518, 1435, 4, 2, 2, 5096, 19266, 2650, 8670, 19, 7193, 16, 4096, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [29]:
# test 데이터셋
test_dataset = tf_glue_convert_examples_to_features(data['test_mismatched'], huggingface_tokenizer, max_length=256, processor=processor)
test_dataset_batch = test_dataset.shuffle(100).batch(16).repeat(2)

Using label list ['contradiction', 'entailment', 'neutral']


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


*** Example ***
guid: 3498
features: InputFeatures(input_ids=[0, 46871, 6, 38, 17888, 14, 2103, 4, 2, 2, 1711, 2103, 34, 57, 11, 127, 284, 13, 6808, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

---
### 4-3 오류
---

    Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.

위의 오류는 토큰을 변환하는 과정에서 오류가 발생한 것으로 파악된다.  
  
- max_len을 설정하는 과정에서 처음에는 128로 설정을 하였다.  
하고 코드를 돌려보니 위 오류의 값이 수도 없이 많이 나왔다.  

- 또한 모델의 val_acc값이 0에 수렴하는 값으로 나왔다. 
해당 원인은 아마 위 오류는 토큰화 하는 과정에서 데이터를 max_len을 기준으로 자르다가 오류가 생겨 공백(Null)값이 생긴것 같은데... 해당 공백으로 인해 accuracy를 측정하는 것에 오류가 발생하지 않았을까 싶다.

- 이후 해당 값을 512으로 설정하여 다시 코드를 돌려보니 위의 오류는 나오지 않았다.  
(512로 설정한 이유는 위 모델의 tokenizer의 max_len값이 512로 설정되어 있다.)
<br><br>
하지만 이후 모델을 돌릴 때, 

    Graph execution error

  오류가 발생하였다. 해당 오류는 max_len의 값이 512로 너무 커서 과부하가 걸려서 나온 것이라 판단되어 해당 값을 256으로 설정하여 모델을 학습을 진행하였다.


---
# 5. 모델 생성 및 학습
---

In [30]:
num_classes = len(processor.get_labels())

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [31]:
huggingface_model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])

huggingface_model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592899    
 ficationHead)                                                   
                                                                 
Total params: 124,647,939
Trainable params: 124,647,939
Non-trainable params: 0
_________________________________________________________________


In [32]:
huggingface_model.fit(train_dataset_batch, epochs=7, steps_per_epoch=115, validation_data=test_dataset_batch)

Epoch 1/7
115/115 [==============================] - 468s 4s/step - loss: 1.1053 - acc: 0.3310 - val_loss: 1.1098 - val_acc: 0.0000e+00
Epoch 2/7
115/115 [==============================] - 447s 4s/step - loss: 0.9592 - acc: 0.5353 - val_loss: 1.6072 - val_acc: 0.1870
Epoch 3/7
115/115 [==============================] - 446s 4s/step - loss: 0.7728 - acc: 0.6886 - val_loss: 1.8291 - val_acc: 0.2867
Epoch 4/7
115/115 [==============================] - 446s 4s/step - loss: 0.6649 - acc: 0.7359 - val_loss: 1.6805 - val_acc: 0.4184
Epoch 5/7
115/115 [==============================] - 446s 4s/step - loss: 0.6812 - acc: 0.7348 - val_loss: 2.2326 - val_acc: 0.2415
Epoch 6/7
115/115 [==============================] - 446s 4s/step - loss: 0.6588 - acc: 0.7266 - val_loss: 2.4888 - val_acc: 0.2781
Epoch 7/7
115/115 [==============================] - 446s 4s/step - loss: 0.6238 - acc: 0.7647 - val_loss: 1.6678 - val_acc: 0.3686


---
# 6. 회고
----

- 노드를 설명하는 과정에서 2가지 방법으로 설명을 해주다보니 다소 헷갈렸던 부분들이 있었다.

- 과정을 진행하면서 생각보다 많은 오류에 직면하게 되었다.  
4-3에서 설명한 오류들은 물론이고 hugging_face에서 모델을 불러오는 과정에서도 오류가 있었다. 

- 처음 xlm-roberta-base 모델을 불러와 학습을 진행하고 싶었다.

- 모델을 불러오는 과정에서 Autoclass를 사용하려 했으며, 모델을 불러오는 것 까지는 잘 되었다.  
하지만 학습하는 과정에서 예상 학습시간이 18시간 정도가 나오는 것을 보았다.  

- 어떤 설정 오류로 인해 그렇게 진행이 된건지는 알 수 없어 시간을 많이 허비했다. 

- 위 오류로 인해 해당 모델을 Autoclass가 아닌 직접 설정하는 과정을 거쳤고, 학습하는 과정에서 오류가 발생하였다. 

- 오류의 원인은 아마 파이토치로 활용해야 하는 모델이었던 것 같은데 파이토치에 대한 이해가 부족해 옳바른 원인 유추와 해결을 진행하지 못 했다.